---
title: "ML Systems & Deployment: From Research to Production"
author: "Ram Polisetti"
date: "2024-03-19"
categories: [machine-learning, deployment, mlops, system-design]
image: "ml_systems.jpg"
description: "A comprehensive guide to designing, deploying, and maintaining machine learning systems in production environments."
jupyter: python3
---




# Machine Learning Systems and Deployment

Moving from experimental notebooks to production systems is one of the most challenging aspects of machine learning. This post covers the essential aspects of building and deploying ML systems.

## 1. ML System Architecture

### Basic ML Service Architecture


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib
import json
from datetime import datetime
import logging
from typing import Dict, List, Union, Any

class MLService:
    def __init__(self, model_path: str = None):
        self.model = None
        self.scaler = None
        self.feature_names = None
        self.model_metadata = {
            'last_training_date': None,
            'model_version': None,
            'performance_metrics': {}
        }
        
        if model_path:
            self.load_model(model_path)
    
    def preprocess_data(self, data: pd.DataFrame) -> np.ndarray:
        """Preprocess input data."""
        if self.scaler is None:
            self.scaler = StandardScaler()
            self.scaler.fit(data)
        return self.scaler.transform(data)
    
    def train(self, X: pd.DataFrame, y: pd.Series) -> Dict[str, float]:
        """Train the model and return performance metrics."""
        # Store feature names
        self.feature_names = list(X.columns)
        
        # Preprocess data
        X_scaled = self.preprocess_data(X)
        
        # Initialize and train model
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.model.fit(X_scaled, y)
        
        # Update metadata
        self.model_metadata.update({
            'last_training_date': datetime.now().isoformat(),
            'model_version': '1.0.0',
            'feature_names': self.feature_names
        })
        
        return {'status': 'success', 'message': 'Model trained successfully'}
    
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """Make predictions on new data."""
        if self.model is None:
            raise ValueError("Model not trained. Please train the model first.")
        
        # Validate features
        if not all(feat in X.columns for feat in self.feature_names):
            raise ValueError("Missing required features in input data.")
        
        # Preprocess and predict
        X_scaled = self.preprocess_data(X[self.feature_names])
        return self.model.predict(X_scaled)
    
    def save_model(self, path: str) -> None:
        """Save model and associated metadata."""
        if self.model is None:
            raise ValueError("No model to save.")
        
        model_artifacts = {
            'model': self.model,
            'scaler': self.scaler,
            'metadata': self.model_metadata
        }
        joblib.dump(model_artifacts, path)
        
    def load_model(self, path: str) -> None:
        """Load model and associated metadata."""
        model_artifacts = joblib.load(path)
        self.model = model_artifacts['model']
        self.scaler = model_artifacts['scaler']
        self.model_metadata = model_artifacts['metadata']
        self.feature_names = self.model_metadata.get('feature_names')

# Example usage
from sklearn.datasets import make_classification

# Generate sample data
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15,
                         n_redundant=5, random_state=42)
feature_names = [f'feature_{i}' for i in range(X.shape[1])]
X_df = pd.DataFrame(X, columns=feature_names)

# Initialize and train service
ml_service = MLService()
training_result = ml_service.train(X_df, y)
print("Training Result:", training_result)

# Make predictions
sample_data = X_df.head(5)
predictions = ml_service.predict(sample_data)
print("\nSample Predictions:", predictions)

# Save and load model
ml_service.save_model('model.joblib')
new_service = MLService('model.joblib')

### Model Versioning and Metadata


In [ ]:
class ModelRegistry:
    def __init__(self):
        self.models = {}
        self.current_version = None
    
    def register_model(self, model_path: str, version: str,
                      metadata: Dict[str, Any]) -> None:
        """Register a new model version."""
        self.models[version] = {
            'path': model_path,
            'metadata': metadata,
            'registration_time': datetime.now().isoformat()
        }
        
    def get_model(self, version: str = None) -> MLService:
        """Retrieve a specific model version."""
        version = version or self.current_version
        if version not in self.models:
            raise ValueError(f"Model version {version} not found.")
        
        return MLService(self.models[version]['path'])
    
    def set_current_version(self, version: str) -> None:
        """Set the current production model version."""
        if version not in self.models:
            raise ValueError(f"Model version {version} not found.")
        self.current_version = version
    
    def list_versions(self) -> pd.DataFrame:
        """List all registered model versions."""
        versions_data = []
        for version, info in self.models.items():
            data = {
                'version': version,
                'path': info['path'],
                'registration_time': info['registration_time']
            }
            data.update(info['metadata'])
            versions_data.append(data)
        
        return pd.DataFrame(versions_data)

# Example usage
registry = ModelRegistry()

# Register model versions
registry.register_model(
    'model.joblib',
    '1.0.0',
    {'accuracy': 0.95, 'environment': 'production'}
)

registry.register_model(
    'model.joblib',
    '1.1.0',
    {'accuracy': 0.96, 'environment': 'staging'}
)

# List versions
print("Registered Models:")
print(registry.list_versions())

## 2. Model Monitoring and Logging

### Performance Monitoring System


In [ ]:
class ModelMonitor:
    def __init__(self):
        self.metrics_log = []
        self.prediction_log = []
        
    def log_prediction(self, input_data: pd.DataFrame, prediction: np.ndarray,
                      metadata: Dict[str, Any]) -> None:
        """Log a prediction event."""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'input_shape': input_data.shape,
            'prediction_shape': prediction.shape,
            'metadata': metadata
        }
        self.prediction_log.append(log_entry)
    
    def log_metrics(self, metrics: Dict[str, float]) -> None:
        """Log performance metrics."""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'metrics': metrics
        }
        self.metrics_log.append(log_entry)
    
    def get_metrics_summary(self) -> pd.DataFrame:
        """Get summary of logged metrics."""
        return pd.DataFrame(self.metrics_log)
    
    def get_prediction_summary(self) -> pd.DataFrame:
        """Get summary of prediction logs."""
        return pd.DataFrame(self.prediction_log)

class MonitoredMLService(MLService):
    def __init__(self, model_path: str = None):
        super().__init__(model_path)
        self.monitor = ModelMonitor()
    
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """Make predictions with monitoring."""
        predictions = super().predict(X)
        
        # Log prediction event
        self.monitor.log_prediction(
            X,
            predictions,
            {'model_version': self.model_metadata.get('model_version')}
        )
        
        return predictions

# Example usage
monitored_service = MonitoredMLService()
monitored_service.train(X_df, y)

# Make predictions with monitoring
for _ in range(5):
    sample_data = X_df.sample(n=10)
    predictions = monitored_service.predict(sample_data)

# View monitoring logs
print("\nPrediction Logs:")
print(monitored_service.monitor.get_prediction_summary())

### Data Drift Detection


In [ ]:
from scipy import stats

class DriftDetector:
    def __init__(self, reference_data: pd.DataFrame):
        self.reference_data = reference_data
        self.reference_statistics = self._compute_statistics(reference_data)
    
    def _compute_statistics(self, data: pd.DataFrame) -> Dict[str, Dict[str, float]]:
        """Compute summary statistics for each feature."""
        stats_dict = {}
        for column in data.columns:
            stats_dict[column] = {
                'mean': data[column].mean(),
                'std': data[column].std(),
                'median': data[column].median(),
                'q1': data[column].quantile(0.25),
                'q3': data[column].quantile(0.75)
            }
        return stats_dict
    
    def detect_drift(self, new_data: pd.DataFrame,
                    threshold: float = 0.05) -> Dict[str, Any]:
        """Detect drift in new data compared to reference data."""
        drift_results = {}
        
        for column in self.reference_data.columns:
            # Perform Kolmogorov-Smirnov test
            ks_statistic, p_value = stats.ks_2samp(
                self.reference_data[column],
                new_data[column]
            )
            
            # Compare statistics
            ref_stats = self.reference_statistics[column]
            new_stats = self._compute_statistics(new_data)[column]
            
            # Calculate relative changes
            stat_changes = {
                stat: abs(new_stats[stat] - ref_stats[stat]) / abs(ref_stats[stat])
                for stat in ref_stats.keys()
            }
            
            drift_results[column] = {
                'ks_statistic': ks_statistic,
                'p_value': p_value,
                'has_drift': p_value < threshold,
                'stat_changes': stat_changes
            }
        
        return drift_results

# Example usage
# Split data into reference and new
reference_data = X_df.iloc[:800]
new_data = X_df.iloc[800:]

# Initialize drift detector
drift_detector = DriftDetector(reference_data)

# Detect drift
drift_results = drift_detector.detect_drift(new_data)

# Display results
print("\nDrift Detection Results:")
for feature, results in drift_results.items():
    if results['has_drift']:
        print(f"\nDrift detected in {feature}:")
        print(f"p-value: {results['p_value']:.4f}")
        print("Statistical changes:")
        for stat, change in results['stat_changes'].items():
            print(f"  {stat}: {change:.2%}")

## 3. Model Serving Infrastructure

### REST API Service


In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Dict

# Define data models
class PredictionRequest(BaseModel):
    features: List[List[float]]
    feature_names: List[str]

class PredictionResponse(BaseModel):
    predictions: List[int]
    model_version: str
    prediction_time: str

# Initialize FastAPI app
app = FastAPI(title="ML Model Service")

# Initialize ML service
ml_service = MonitoredMLService()

@app.post("/predict", response_model=PredictionResponse)
async def predict(request: PredictionRequest):
    try:
        # Convert input to DataFrame
        input_data = pd.DataFrame(
            request.features,
            columns=request.feature_names
        )
        
        # Make prediction
        predictions = ml_service.predict(input_data)
        
        return PredictionResponse(
            predictions=predictions.tolist(),
            model_version=ml_service.model_metadata['model_version'],
            prediction_time=datetime.now().isoformat()
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Note: In a real application, you would run this with uvicorn
print("\nAPI Documentation:")
print("POST /predict")
print("Request body schema:", PredictionRequest.schema())
print("Response body schema:", PredictionResponse.schema())

## 4. Deployment Strategies

### A/B Testing Implementation


In [ ]:
class ABTestingService:
    def __init__(self, model_a: MLService, model_b: MLService,
                 traffic_split: float = 0.5):
        self.model_a = model_a
        self.model_b = model_b
        self.traffic_split = traffic_split
        self.results = {
            'model_a': {'predictions': 0, 'errors': 0},
            'model_b': {'predictions': 0, 'errors': 0}
        }
    
    def predict(self, X: pd.DataFrame) -> Dict[str, Any]:
        """Make prediction using A/B testing."""
        # Randomly select model based on traffic split
        if np.random.random() < self.traffic_split:
            model = self.model_a
            model_name = 'model_a'
        else:
            model = self.model_b
            model_name = 'model_b'
        
        try:
            prediction = model.predict(X)
            self.results[model_name]['predictions'] += 1
            
            return {
                'prediction': prediction,
                'model_used': model_name,
                'model_version': model.model_metadata['model_version']
            }
        
        except Exception as e:
            self.results[model_name]['errors'] += 1
            raise e
    
    def get_statistics(self) -> Dict[str, Any]:
        """Get A/B testing statistics."""
        return {
            'model_a_stats': {
                'usage_percentage': self.results['model_a']['predictions'] /
                    max(1, sum(r['predictions'] for r in self.results.values())),
                'error_rate': self.results['model_a']['errors'] /
                    max(1, self.results['model_a']['predictions'])
            },
            'model_b_stats': {
                'usage_percentage': self.results['model_b']['predictions'] /
                    max(1, sum(r['predictions'] for r in self.results.values())),
                'error_rate': self.results['model_b']['errors'] /
                    max(1, self.results['model_b']['predictions'])
            }
        }

# Example usage
# Create two model versions
model_a = MLService()
model_b = MLService()

# Train models with slightly different data
model_a.train(X_df.iloc[:800], y[:800])
model_b.train(X_df.iloc[200:], y[200:])

# Initialize A/B testing service
ab_service = ABTestingService(model_a, model_b, traffic_split=0.7)

# Make predictions
for _ in range(100):
    sample_data = X_df.sample(n=1)
    try:
        result = ab_service.predict(sample_data)
    except Exception:
        continue

# Get statistics
print("\nA/B Testing Statistics:")
print(json.dumps(ab_service.get_statistics(), indent=2))

## Best Practices for ML Systems

1. **System Design**
   - Modular architecture
   - Clear separation of concerns
   - Proper error handling
   - Comprehensive logging

2. **Deployment**
   - Automated testing
   - Gradual rollout
   - Monitoring and alerting
   - Rollback capabilities

3. **Maintenance**
   - Regular model updates
   - Performance monitoring
   - Data drift detection
   - Resource optimization

4. **Documentation**
   - System architecture
   - API specifications
   - Model versioning
   - Deployment procedures

## Common Challenges and Solutions

1. **Scaling Issues**
   - Load balancing
   - Caching strategies
   - Batch processing
   - Resource management

2. **Model Degradation**
   - Regular retraining
   - Performance monitoring
   - Data quality checks
   - Version control

3. **System Reliability**
   - Redundancy
   - Error handling
   - Failover strategies
   - Backup procedures

## Conclusion

Building production ML systems requires:

1. Robust architecture
2. Comprehensive monitoring
3. Effective deployment strategies
4. Proper maintenance procedures

In the next post, we'll explore specific industry applications and case studies.

## Additional Resources

1. Books:
   - "Designing Machine Learning Systems" by Chip Huyen
   - "Machine Learning Design Patterns" by Lakshmanan et al.
   - "Building Machine Learning Powered Applications" by Emmanuel Ameisen

2. Online Resources:
   - MLOps Community
   - Google Cloud ML Operations
   - AWS ML Deployment Best Practices

Remember: A well-designed ML system is more than just a model—it's a complete solution that handles data processing, monitoring, and maintenance effectively.